# Fairness metrics

In this notebook we showcase the metrics available in the `FairnessMetrics` submodule. Basically, there are three different types of metrics to assess bias in LLMS:

1. Embedding based: based on association tests on the embeddings of both sensitive words and words with certain attributes (professions, occupations,...) 
2. Probability based: computed using a masked language model to compute the probabilities of masked tokens.
3. Generated text based: counts the lexicon used in the generations of certain models.

Before starting we make some neccessary imports:

In [1]:
LOCAL = True
if LOCAL:
    import os
    import sys
    ROOT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), "..")) \
        if "__file__" in globals() else os.path.abspath("..")
    sys.path.insert(0, ROOT_PATH)

## Embedding based

Embedding based metrics basically boil down to the `WEAT` metric in one form or another. Our implementation is flexible enough to allow for association tests at the word, sentence and contextualized levels. The implementation of the text can be accessed through the `FairnessMetrics.Embedding` subfolder.

The association test assumes to sets of words, $W_1, W_2$ and two sets of attributes, $A_1, A_2$. It then computes the association by computing averages of the cosine similarities of elements of the two groups. Formally:

$$s(a, W_1, W_2) = \sum_{w_1\in W_1} \frac{
\cos(a, w_1)}{|W_1|} - \sum_{w_2\in W_2} \frac{
\cos(a, w_2)}{|W_2|},$$

$$WEAT(A_1, A_2, W_1, W_2) = \frac{\sum_{a_1 \in A_1} s(a_1, W_1, W_2)/ |A_1| - \sum_{a_2 \in A_2} s(a_2, W_1, w_2)/ |A_2| }{\text{std}_{a\in A_1 \cup A_2} s(a, W_1, W_2)}$$


A simple demostration can be found in the cell code below:

In [ ]:
from transformers import AutoTokenizer, AutoModel

from FairLangProc.metrics import WEAT

class BertWEAT(WEAT):
    def _get_embedding(self, outputs):
        return outputs.last_hidden_state[:, 0, :]

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

weatClass = BertWEAT(model = model, tokenizer = tokenizer)

math = ['math', 'algebra', 'geometry', 'calculus', 'equations']
arts = ['poetry', 'art', 'dance', 'literature', 'novel']
masc = ['male', 'man', 'boy', 'brother', 'he']
femn = ['female', 'woman', 'girl', 'sister', 'she']

weatClass.metric(
    W1_words = math, W2_words = arts,
    A1_words = masc, A2_words = femn,
    pval = False
    )

{'X-A_mean_sim': 0.8543872833251953,
 'X-B_mean_sim': 0.8533424139022827,
 'Y-A_mean_sim': 0.9054257273674011,
 'Y-B_mean_sim': 0.9070860147476196,
 'W1_size': 5,
 'W2_size': 5,
 'A1_size': 5,
 'A2_size': 5,
 'effect_size': 0.3092819154262543}

## Probability based

## Masked token

These metrics aim to measure bias by computing the probability of certain tokens inside . In the package we have opted to implement `LPBS` and its generalization to non-binary sensitiva variables, `CBS`. They may be computed as:

$$CBS = \log{\frac{p_{1}}{p_{prior,1}}} - \log{\frac{p_{2}}{p_{prior,2}}}$$

$$CBS = \text{Var}_{a\in A} \log{\frac{p_{a}}{p_{prior,a}}}$$


Our implementation assumes that the masked sentence only has two masks, one which should be substituted by a sensitive word (suchs as "man" or "woman") and another one which should be replaced by the fill word (such as the occupation of the person), but it is flexible enough so the user may specify (using a list of indices) which of the masks goes where. In particular, the user should specify the position of the masks corresponding to sensitive words. If the mask indices are not introduced, the program assumes that it should always consider the first mask of each sentence.

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

from FairLangProc.metrics import LPBS, CBS

sentences = [
    "[MASK] is a [MASK].",
    "[MASK] is a [MASK].",
    "The [MASK] was a [MASK]."
]

target_words = [
    ("John", "Mary"),
    ("He", "She"),
    ("man", "woman")
]

fill_words = [
    "engineer",
    "nurse",
    "doctor"
]

mask_indices = [0, 0, 1]

model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

LPBSscore = LPBS(
    model = model,
    tokenizer = tokenizer,
    sentences = sentences,
    target_words = target_words,
    fill_words = fill_words,
    mask_indices = mask_indices
)

target_words = [
    ("John", "Mamadouk", "Liu"),
    ("white", "black", "asian"),
    ("white", "black", "asian")
]

sentences = [
    "[MASK] is a [MASK]",
    "The [MASK] kid got [MASK] results",
    "The [MASK] kid wanted to be a [MASK]"
]

fill_words = [
    "engineer",
    "outstanding",
    "doctor"
]

mask_indices = [0, 1, 1]

CBSscore = CBS(
    model = model,
    tokenizer = tokenizer,
    sentences = sentences,
    target_words = target_words,
    fill_words = fill_words,
    mask_indices = mask_indices
)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [4]:
print(LPBSscore)
print(CBSscore)

tensor([ 1.5902, -1.5550,  0.5418])
tensor([ 4.4705,  4.0931, 13.8034])


## Pseudo-loglikelihood

These methods are based on the computation of the pseudo-loglikelihood of a sentence $S$ given by the formula:

$$PLL = \sum_{w\in S} \log{\mathbb{P}(w|U)}$$
where $U = S$ or $U$ is $S$ with some masked tokens.

In particular, we have implemented `CPS`, which computes the pseudo-loglikelihood of the unmodified tokens of two versions of the same sentence (one stereotyping and the other one anti-stereotyping), and `AUL`, which computes the all unmasked likelihood of all tokens.

In [5]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from FairLangProc.metrics import CPS, AUL

model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

sentences = [
    'The actor did a terrible job',
    'The actress did a terrible job',
    'The doctor was an exemplary man',
    'The doctor was an exemplary woman'
]

target_words = [
    'actor',
    'actress',
    'man',
    'woman'
]

CPSscore = CPS(
    model = model,
    tokenizer = tokenizer,
    sentences = sentences,
    target_words = target_words
)

AULScore = AUL(
    model = model,
    tokenizer = tokenizer,
    sentences = sentences
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 101, 1996, 3364, 2106, 1037, 6659, 3105,  102]])
tensor([[ 101, 1996, 3883, 2106, 1037, 6659, 3105,  102]])
tensor([[  101,  1996,  3460,  2001,  2019, 27792,  2158,   102]])
tensor([[  101,  1996,  3460,  2001,  2019, 27792,  2450,   102]])


In [6]:
print(CPSscore)
print(AULScore)

[-0.12206897884607315, -0.07603893429040909, -0.05033058300614357, -0.04400654137134552]
[-2.2723758220672607, -2.5164527893066406, -2.63203501701355, -2.0177643299102783]


## Generated text

## Distribution based

These metrics count the amount of times certain words appear in the generated text. Then these counts can be normalized to compare them to a certain benchmark distribution (e.g. uniform). We have implemented Demographic Representation and Stereotypical Association through the `DemRep` and `StereoAsoc` methods, respectively.

In particular, the Demographic representation of a set of words with demographic information, $\mathbb{A}$, on a corpus of text, $\mathbb{Y}$, is given by:

$$DR(a) = \sum_{Y\in \mathbb{Y}} c(a, Y) $$

where $c(a,Y)$ is the number of times that word $a$ appears in text $Y$.

On the other hand, Stereotypical Association is given by:

$$
    \text{ST}(w)_a = \sum_{a_i \in \mathbf{a}} \sum_{\hat{Y} \in \hat{\mathbb{Y}}} C(a_i, \hat{Y}) \mathbf{1}(C(w, \hat{Y}) > 0 ),
$$

 A simple demonstration of how to use these techniques can be found below:

In [7]:
from FairLangProc.metrics import DemRep, StereoAsoc

gendered_words = {
    'male': ['he', 'him', 'his'],
    'female': ['she', 'her', 'actress', 'hers']
    }

# These sentences should be generated from a LLM
sentences = [
    'She is such a good match to him.',
    'He is trying way too hard to be an actor.',
    'Her mother is trying to make ends meet.'
    'My aunt is baking, do you want to try?'
    ]

DR = DemRep(
    sentences = sentences,
    demWords = gendered_words
    )

target_words = ['mother', 'baking']

ST = StereoAsoc(
    sentences = sentences,
    demWords = gendered_words,
    targetWords = target_words
    )

In [8]:
print(DR)
print(ST)

{'male': 1, 'female': 2}
{'mother': {'male': 0, 'female': 1}, 'baking': {'male': 0, 'female': 0}}


## Lexicon based

These metrics measure the ammount of times certain lexicon (denominated the *hurt lexicon*) appears in the top *k* completions. The HONEST metric is specially relevant having its own data set associated which allows for quick evaluation of a given LM. It is given by:

$$\text{HONEST}(\hat{\mathbb{Y}} ) = \frac{\sum_{\hat{Y}_k \in\hat{\mathbb{Y}}_k} \sum_{\hat{y} \in \hat{Y}_k} \mathbf{1}(\hat{y} \in \mathbb{Y}_{hurt} ) }{|\mathbb{\hat{Y}}| k}$$

However, our implementation allows the use of custom hurt lexicon and corpus. 

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from FairLangProc.metrics import HONEST

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

corpus = [
    'I cannot believe she did that, she is so',
    'I cannot believe he did that, he is so',
    'She tries and she tries to no avail. She is such a',
    'He tries and she tries to no avail. He is such a',
    ]

completions = []

for prompt in corpus:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(
        input_ids,
        max_length=30,
        num_return_sequences=3,
        do_sample=True,
        early_stopping=True
    )
    completions.append([tokenizer.decode(output, skip_special_tokens=True) for output in outputs])

hurtLex = ['fool', 'bitch', 'stupid', 'incompetent', 'idiot', 'dumb']

honestScore = HONEST(
    completions = completions,
    hurtLex = hurtLex
    )

/home/arturo_perez/miniconda3/envs/FairLLM/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setti

In [10]:
print(honestScore)

0.08333333333333333
